In [1]:
import os
import time
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

class GDA:
    def __init__(self, verbose=True):
        self.theta = None
        self.theta_0 = None
        self.verbose = verbose
    
    def fit(self, x, y):
        n_examples, dim = x.shape[0], x.shape[1]  
        phi = np.sum(y) / n_examples 
        mu0 = np.mean(x[y == 0], axis=0) if np.sum(y == 0) > 0 else np.zeros(dim)
        mu1 = np.mean(x[y == 1], axis=0) if np.sum(y == 1) > 0 else np.zeros(dim)
        sigma = np.zeros((dim, dim)) 
        for i in range(n_examples): 
            mu = mu0 if y[i] == 0 else mu1
            sigma += np.outer((x[i] - mu), (x[i] - mu))
        sigma = sigma / n_examples if n_examples > 0 else np.eye(dim)
        sigma_inv = np.linalg.inv(sigma + 1e-3 * np.eye(dim))
        self.theta = sigma_inv @ (mu1 - mu0)
        self.theta_0 = (
            -0.5 * mu1.T @ sigma_inv @ mu1
            + 0.5 * mu0.T @ sigma_inv @ mu0
            + np.log((phi + 1e-5) / (1 - phi + 1e-5))
        )
        self.theta = np.concatenate([[self.theta_0], self.theta])

    def predict(self, x):
        z = np.clip(x @ self.theta + self.theta_0, -100, 100)
        return 1 / (1 + np.exp(-z))

dataset_path = "../../CNNExpirement/dataset"
classes = {"Strawberry___Leaf_scorch": 0, "Strawberry__healthy": 1}
IMG_SIZE = (64, 64)

def load_images():
    X, y = [], []
    for class_name, label in classes.items():
        folder_path = os.path.join(dataset_path, class_name)
        if not os.path.exists(folder_path):
            continue
        for file in os.listdir(folder_path):
            if file.lower().endswith((".jpeg", ".jpg", ".png")):
                img_path = os.path.join(folder_path, file)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                img = cv2.resize(img, IMG_SIZE)
                img = img / 255.0
                X.append(img.flatten())
                y.append(label)
    X, y = np.array(X), np.array(y)
    if X.size == 0 or y.size == 0:
        raise ValueError("No images were loaded.")
    return X, y

X, y = load_images()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

start_time = time.time()
clf = GDA()
clf.fit(X_train, y_train)
training_time = time.time() - start_time

X_test_with_bias = np.hstack([np.ones((X_test.shape[0], 1)), X_test])
y_pred = clf.predict(X_test_with_bias) > 0.5
accuracy = accuracy_score(y_test, y_pred)

print(f"Training Time: {training_time:.4f} seconds")
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Training Time: 129.6333 seconds
Model Accuracy: 16.99%
